In [3]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier 


## Examine the Data

The data needs to be examined to clean and wrangle what we need from it. 

In [4]:
# Load Data and get information on it. 
df = pd.read_csv("data/complaints.csv", low_memory=False)
print(f"The data frame is {df.size} its' shape is {df.shape}, This is large so we need to do some cleaning and wrangling.")


The data frame is 292356715 its' shape is (8353049, 35), This is large so we need to do some cleaning and wrangling.


## Here is a definition table of the columns. 

Metadata definitions are imperative to understand so that data can be analyzed effectively. After reviewing the data [here](https://data.cityofnewyork.us/api/views/57mv-nv28/files/5355df15-d454-486c-9796-6f894e311ef7?download=true&filename=NYPDIncidentLevelDataFootnotes.pdf) and [here](https://data.cityofnewyork.us/api/views/57mv-nv28/files/cd8fdc0b-e645-4a37-8cc1-27d95e7666b7?download=true&filename=NYPD_Incident_Level_Data_Column_Descriptions.csv), I have created a table of all of the metadata and actions I may want to utilize.


| No. | Column            | Action                      | Potential Use          | Rationale                                                         |
|-----|-------------------|-----------------------------|------------------------|-------------------------------------------------------------------|
| 1   | CMPLNT_NUM        | Keep                        | Identifier             | Unique identifier for each complaint, useful for record tracking. |
| 2   | CMPLNT_FR_DT      | Merge with CMPLNT_FR_TM     | Temporal analysis      | Combined with time for precise datetime of incidents.             |
| 3   | CMPLNT_FR_TM      | Merge with CMPLNT_FR_DT     | Temporal analysis      | Combined with date for precise datetime of incidents.             |
| 4   | CMPLNT_TO_DT      | Merge with CMPLNT_TO_TM     | Temporal analysis      | Ending datetime of incidents can be useful for duration analysis. |
| 5   | CMPLNT_TO_TM      | Merge with CMPLNT_TO_DT     | Temporal analysis      | Ending datetime of incidents can be useful for duration analysis. |
| 6   | RPT_DT            | Drop                        | -                      | Report date may not directly impact fare safety.                  |
| 7   | KY_CD             | Keep                        | Categorical analysis   | Crime type can be relevant for assessing danger levels.           |
| 8   | OFNS_DESC         | Keep                        | Categorical analysis   | Descriptive analysis of offense types.                            |
| 9   | PD_CD             | Keep                        | Categorical analysis   | Granular details on offenses can help refine danger assessments.  |
| 10  | PD_DESC           | Keep                        | Categorical analysis   | Detailed descriptions provide context for ML models.              |
| 11  | CRM_ATPT_CPTD_CD  | Keep                        | Binary classification  | Indicator of crime completion may correlate with danger levels.   |
| 12  | LAW_CAT_CD        | Keep                        | Categorical analysis   | Severity of crime can be a strong indicator of danger.            |
| 13  | JURIS_DESC        | Drop                        | -                      | Jurisdiction may not be relevant for fare safety analysis.        |
| 14  | BORO_NM           | Keep                        | Geospatial analysis    | Borough name can be used for location-based analysis.             |
| 15  | ADDR_PCT_CD       | Keep                        | Geospatial analysis    | Precinct info can help in location-based danger assessment.       |
| 16  | LOC_OF_OCCUR_DESC | Keep                        | Categorical/Text analysis | Location description can provide context to the incident.     |
| 17  | PREM_TYP_DESC     | Keep                        | Categorical/Text analysis | Premises type can be relevant for understanding crime context. |
| 18  | PARKS_NM          | Drop                        | -                      | Specific park names may not be relevant for fare safety.          |
| 19  | HADEVELOPT        | Drop                        | -                      | Housing development names may not be directly relevant.           |
| 20  | X_COORD_CD        | Keep                        | Geospatial analysis    | Coordinates can be crucial for mapping crime locations.           |
| 21  | Y_COORD_CD        | Keep                        | Geospatial analysis    | Coordinates can be crucial for mapping crime locations.           |
| 22  | Latitude          | Keep                        | Geospatial analysis    | Latitude for mapping and proximity analysis.                      |
| 23  | Longitude         | Keep                        | Geospatial analysis    | Longitude for mapping and proximity analysis.                     |
| 24  | SUSP_AGE_GROUP    | Drop                        | -                      | Suspect's age may not significantly impact fare safety.           |
| 25  | SUSP_RACE         | Drop                        | -                      | Suspect's race may not be relevant and poses ethical concerns.    |
| 26  | SUSP_SEX          | Drop                        | -                      | Suspect's sex may not be directly relevant to fare safety.        |
| 27  | TRANSIT_DISTRICT  | Drop                        | -                      | Transit district may not be directly relevant for rideshare/taxi analysis. |
| 28  | Latitude           | Keep (Duplicate)             | Geospatial analysis                            | Duplicate, but necessary for geospatial analysis. |
| 29  | Longitude          | Keep (Duplicate)             | Geospatial analysis                            | Duplicate, but necessary for geospatial analysis. |
| 30  | Lat_Lon            | Drop                         | -                                              | Redundant with separate latitude and longitude columns. |
| 31  | PATROL_BORO        | Keep                         | Geospatial analysis                            | Patrol borough can provide additional geospatial context. |
| 32  | STATION_NAME       | Drop                         | -                                              | Station names may not be directly relevant for fare safety. |
| 33  | VIC_AGE_GROUP      | Drop                         | -                                              | Victim's age may not significantly impact fare safety. |
| 34  | VIC_RACE           | Drop                         | -                                              | Victim's race may not be relevant and poses ethical concerns. |
| 35  | VIC_SEX            | Drop                         | -                                              | Victim's sex may not be directly relevant to fare safety. |


## Take Initial Actions

In [5]:
# Check for out-of-range or invalid dates
invalid_dates = pd.to_datetime(df['CMPLNT_FR_DT'], errors='coerce').isna().sum()
invalid_times = pd.to_datetime(df['CMPLNT_FR_TM'], format='%H:%M:%S', errors='coerce').isna().sum()

print(f"Invalid dates in CMPLNT_FR_DT: {invalid_dates} ({(invalid_dates / len(df) * 100):.2f}%)")
print(f"Invalid times in CMPLNT_FR_TM: {invalid_times} ({(invalid_times / len(df) * 100):.2f}%)")

# Optionally, you can also see some of the problematic entries
print("Some problematic date entries:")
print(df[df['CMPLNT_FR_DT'].isna() | df['CMPLNT_FR_TM'].isna()][['CMPLNT_FR_DT', 'CMPLNT_FR_TM']].head())


Invalid dates in CMPLNT_FR_DT: 796 (0.01%)
Invalid times in CMPLNT_FR_TM: 48 (0.00%)
Some problematic date entries:
        CMPLNT_FR_DT CMPLNT_FR_TM
5083246          NaN     08:15:00
5083247          NaN     12:15:00
5083265          NaN     01:45:00
5083283          NaN     11:57:00
5083306          NaN     14:00:00


In [6]:
# Drop the messed up dates:
df = df.dropna(subset=['CMPLNT_FR_DT', 'CMPLNT_FR_TM'])

In [7]:
# Merge CMPLNT_FR_DT and CMPLNT_FR_TM
df['CMPLNT_FR_DT'] = pd.to_datetime(df['CMPLNT_FR_DT'] + ' ' + df['CMPLNT_FR_TM'], errors='coerce')
df.drop('CMPLNT_FR_TM', axis=1, inplace=True)

# Merge CMPLNT_TO_DT and CMPLNT_TO_TM
df['CMPLNT_TO_DT'] = pd.to_datetime(df['CMPLNT_TO_DT'] + ' ' + df['CMPLNT_TO_TM'], errors='coerce')
df.drop('CMPLNT_TO_TM', axis=1, inplace=True)

# Select the Year 2022
df = df[df['CMPLNT_FR_DT'].dt.year == 2022]


In [8]:
# Calculate missing values and their percentage
missing_values = df.isnull().sum()
percentage_missing = (missing_values / len(df)) * 100
most_common_dtype = df.dtypes.apply(lambda x: x.name)

# Determine actions based on the percentage
def determine_action(percentage):
    if percentage > 50:
        return "Consider Dropping"
    elif 0 < percentage <= 50:
        return "Potential Imputation"
    else:
        return "No Action Needed"

# Creating a summary DataFrame
missing_summary = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': percentage_missing.round(2),
    'Action': percentage_missing.apply(determine_action),
    'Most Common Type': most_common_dtype
})

# Sorting the table from most to least missing values
missing_summary.sort_values(by='Missing Values', ascending=False, inplace=True)

print(missing_summary)


                   Missing Values  Percentage (%)                Action  \
TRANSIT_DISTRICT           509575           97.87     Consider Dropping   
CMPLNT_TO_DT                38067            7.31  Potential Imputation   
PD_CD                         410            0.08  Potential Imputation   
ADDR_PCT_CD                    26            0.00  Potential Imputation   
Lat_Lon                         3            0.00  Potential Imputation   
Longitude                       3            0.00  Potential Imputation   
Y_COORD_CD                      3            0.00  Potential Imputation   
Latitude                        3            0.00  Potential Imputation   
X_COORD_CD                      3            0.00  Potential Imputation   
HOUSING_PSA                     0            0.00      No Action Needed   
PATROL_BORO                     0            0.00      No Action Needed   
STATION_NAME                    0            0.00      No Action Needed   
VIC_AGE_GROUP            

In [9]:
crime_categories = df['OFNS_DESC'].unique()
print(crime_categories)

['RAPE' 'ROBBERY' 'MURDER & NON-NEGL. MANSLAUGHTER'
 'ASSAULT 3 & RELATED OFFENSES' 'FELONY ASSAULT'
 'VEHICLE AND TRAFFIC LAWS' 'CRIMINAL MISCHIEF & RELATED OF'
 'UNAUTHORIZED USE OF A VEHICLE' 'GRAND LARCENY' 'DANGEROUS DRUGS'
 'BURGLARY' 'PETIT LARCENY' 'GRAND LARCENY OF MOTOR VEHICLE'
 'HARRASSMENT 2' 'INTOXICATED & IMPAIRED DRIVING' 'SEX CRIMES'
 'CRIMINAL TRESPASS' 'MISCELLANEOUS PENAL LAW'
 'OFF. AGNST PUB ORD SENSBLTY &' 'FRAUDS' 'DANGEROUS WEAPONS'
 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED' 'ALCOHOLIC BEVERAGE CONTROL LAW'
 'ADMINISTRATIVE CODE' 'THEFT-FRAUD' 'FORGERY'
 'NYS LAWS-UNCLASSIFIED FELONY' 'OFFENSES AGAINST THE PERSON'
 'OFFENSES AGAINST PUBLIC ADMINI' 'THEFT OF SERVICES'
 'POSSESSION OF STOLEN PROPERTY' 'OFFENSES INVOLVING FRAUD'
 'PROSTITUTION & RELATED OFFENSES' 'OTHER OFFENSES RELATED TO THEF'
 'OTHER STATE LAWS (NON PENAL LA' 'ARSON' 'HOMICIDE-NEGLIGENT,UNCLASSIFIE'
 'PETIT LARCENY OF MOTOR VEHICLE' "BURGLAR'S TOOLS" 'FRAUDULENT ACCOSTING'
 'KIDNAPPING & RELATED O

## First Glance We 

In [10]:

print(f"The cleaned data frame size is {df.size}")
print(df.head)
df.to_csv("data/2022complaints.csv")

The cleaned data frame size is 17181813
<bound method NDFrame.head of           CMPLNT_NUM        CMPLNT_FR_DT        CMPLNT_TO_DT  ADDR_PCT_CD  \
96         239164165 2022-01-07 22:00:00                 NaT         47.0   
97         238976313 2022-01-09 09:00:00                 NaT         47.0   
98         244480134 2022-05-02 21:43:00                 NaT         43.0   
99         250098128 2022-08-24 20:45:00                 NaT         43.0   
100      251103859H2 2022-09-12 04:42:00                 NaT          NaN   
...              ...                 ...                 ...          ...   
8353044    261171983 2022-12-30 12:31:00                 NaT        106.0   
8353045    261175492 2022-12-31 11:56:00 2022-12-31 17:08:00         44.0   
8353046    261147482 2022-12-30 17:20:00 2022-12-30 17:28:00         17.0   
8353047    261179651 2022-12-31 20:20:00 2022-12-31 20:26:00        112.0   
8353048    261157928 2022-12-31 04:00:00                 NaT         52.0   

     

In [11]:
# Remove rows with missing values in specified columns
columns_with_missing_values = ['TRANSIT_DISTRICT', 'CMPLNT_TO_DT', 'PD_CD', 
                               'ADDR_PCT_CD', 'Lat_Lon', 'Longitude', 
                               'Y_COORD_CD', 'Latitude', 'X_COORD_CD']
df.dropna(subset=columns_with_missing_values, inplace=True)

# Drop the first two columns
df.drop(df.columns[[0, 1]], axis=1, inplace=True)

In [12]:
print(f"The cleaned data frame size is {df.size}")
df.to_csv("data/2022_final_clean_complaints.csv")

The cleaned data frame size is 328879
